IEBS - Proyecto Final

Este notebook se encarga a traves de las APi de twitter buscar los tweets de los diferentes bancos a ser evaluados. Con esto se genera un archivo csv de los tweets y de los usuarios. 

In [1]:
# Importar el driver para almacenar los datos

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Se importan las librerias a utilizar durante todo el proyecto. 
import requests
from requests_oauthlib import OAuth1
import pandas as pd

from tweepy import OAuthHandler, API, Cursor
import json

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Se toman los valores de las llaves para poder autenticarse con el api

ckey = 'dHJRdt64HiFuM9Pd9JjfRKmaO'
csecret = 'ZeCqoUN9EntyQTWrQ1LjeuhRCcuPJsscQOhjtToR9o0WgolOPJ'

atoken = '38060754-NJ7TvzMaZGTbPqxN47R7Lo7nqonm92UnQ00gRcFLd'
asecret = 'Ce5lMmc97yKsloWxNT6MBCduFj8UcUQUXuWW39lz8Pezk'

Bearer = 'AAAAAAAAAAAAAAAAAAAAAHbEOwEAAAAAmFuxYzwGEgoSPj9UXJID8tJA4A4%3DdHHFgBO1XnM4MWI9Z40Tim28zBieDeTwPhU3EGMvFo6F6tvRcO'

In [5]:
# Se genera una instancia de autenticacion
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

In [6]:
# Se realiza una prueba del api. 
api = API(auth,
        wait_on_rate_limit=True,
        wait_on_rate_limit_notify=True)

# API con request

Request de prueba:
curl "https://api.twitter.com/2/users/2244994945/mentions?expansions=author_id&tweet.fields=conversation_id,lang&user.fields=created_at,entities&max_results=5" -H "Authorization: Bearer $BEARER_TOKEN"

Página: 
https://developer.twitter.com/en/docs/twitter-api/tweets/timelines/api-reference/get-users-id-mentions#tab1


In [7]:
# Se invoca el API que permite buscar a traves del nombre de usuario el ID para las busquedas de las menciones. 

headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
url = f'https://api.twitter.com/2/users/by' # url de invocacion
bancos = '' # variable que guarda toda la respuesta

params = {'usernames': 'BBVA_espana,santander_es,caixabank,BancoSabadell,Bankinter',}

auth = OAuth1(ckey, csecret,
               atoken, asecret)

bancos = requests.get(url, params=params, headers=headers).json()
bancos

{'data': [{'id': '230123496',
   'name': 'BBVA en España',
   'username': 'BBVA_espana'},
  {'id': '1372470686', 'name': 'Santander España', 'username': 'santander_es'},
  {'id': '270429778', 'name': 'CaixaBank', 'username': 'caixabank'},
  {'id': '9980072', 'name': 'Banco Sabadell', 'username': 'BancoSabadell'},
  {'id': '86031609', 'name': 'Bankinter', 'username': 'Bankinter'}]}

In [12]:
# Se crea una funcion para buscar las menciones por ID, esta funcion devuelve dos DF, 
# El primero contiene todos los tweets que mencionan el ID del banco que corresponda. 
# El segundo contiene los usuarios que hacen mencion al ID. 

def buscar_tweets_mentions(mentions_id, cantidad_tweet=800):

    headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
    url = f'https://api.twitter.com/2/users/{mentions_id}/mentions' # url de invocacion
    respuesta = '' # variable que guarda toda la respuesta
    usuarios = pd.DataFrame() # variable que guarda los usuarios
    data = pd.DataFrame()
    params = {
        'expansions': 'author_id,geo.place_id',
        'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
        'max_results':100} 

    auth = OAuth1(ckey, csecret,
               atoken, asecret)
    
    for cantidad in range(int(cantidad_tweet/100)):
        respuesta = requests.get(url, params=params, headers=headers).json()
        data = data.append(respuesta['data'])
        usuarios = usuarios.append(respuesta['includes']['users'])
        params = {
            'expansions': 'author_id,geo.place_id',
            'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
            'pagination_token': respuesta['meta']['next_token'],
            'max_results':100}
    return data, usuarios

In [13]:
mentions_id = 1372470686
headers = {'Authorization': f'Bearer {Bearer}'} # Variable de autorizador
url = f'https://api.twitter.com/2/users/{mentions_id}/mentions' # url de invocacion
respuesta = '' # variable que guarda toda la respuesta
usuarios = pd.DataFrame() # variable que guarda los usuarios
data = pd.DataFrame()
params = {
        'expansions': 'author_id,geo.place_id',
        'tweet.fields': 'conversation_id,created_at,lang,public_metrics,possibly_sensitive,source',
        'max_results':100} 
respuesta = requests.get(url, params=params, headers=headers).json()
data = data.append(respuesta['data'])
usuarios = usuarios.append(respuesta['includes']['users'])


In [14]:
usuarios

,id,name,username
0,2407856840,Diana,SINCOMUNISMOo
1,583131735,Pablo Herrera,RMCFSupporter
2,1240360750113345536,Worst Investor,worstinvestorr
3,3439319698,Javier,xavigarciav3
4,1428813295918718996,RABIGA🌐🇰🇿🇪🇸,RabigaKaz
5,1456196870951297027,Juancar82,Juancar827
6,1442225852532412417,🔥 Pedro_Busquets.xrp🔥🦉,Mr_589_
7,376693468,Áлваро,alvaroperis15
8,489717816,Luís,luismerga76
9,714820264568176641,Padre sin trabajo,padresintrabajo


In [ ]:
#Se invoca la funcion pasando como dato el ID del usuario que previamente fue cargado. 

valores = bancos['data'] # Se toman los valores de la lista
for i in valores:
  id = i['id']
  # Se invoca la llamada a la funcion
  respuesta_funcion, usuarios_funcion = buscar_tweets_mentions(id)
  # Se ajusta la columna de fecha
  respuesta_funcion['created_at'] = pd.to_datetime(respuesta_funcion['created_at'])
  # Se guardan los dos archivos con el nombre del ID
  respuesta_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/{id}.csv')
  usuarios_funcion.to_csv(f'/content/drive/MyDrive/IEBS/Global Proyect/datos/user-{id}.csv')